In [51]:
import pandas as pd
import re
import torch
import numpy as np
import torch
import transformers
from ast import literal_eval
import json
import pandas as pd
import random


pd.set_option('display.max_columns', None)

In [52]:
df1 = pd.read_csv("normal_data2.csv")

In [53]:
df1

,id,paragraph,question,choices,answer,question_plus
0,augmented-0,“19세기말 일본은 폐쇄적인 에도 시대를 지나 메이지 유신을 통해 급속한 근대화를 ...,위 글에서 설명된 일본의 근대화 과정에서 나타난 주요 사회적 문제는 무엇으로 분석될...,"['민주주의 제도의 확립', '농민과 노동자의 경제적 불만', '전통 문화의 부흥'...",2,NaN
1,augmented-1,교육부는 최근 한국의 고등학교 교육과정 개편을 통해 '진로 탐색' 과목을 신설하겠다...,교육부가 신설 예정인 '진로 탐색' 과목의 목적은 무엇인가?,"['학생들이 다양한 직업 세계를 이해하기 위해서', '취업률을 높이기 위해서', '...",1,NaN
2,augmented-2,"“[한국사] 조선시대 중기에는 사대부와 농민 사이의 갈등이 심화되었고, 이는 다양한...",위 글에서 언급된 '대동법'의 시행 배경은 무엇과 관련이 있습니까?,"['양반의 세금 부담 증가', '농민의 경제적 안정 추구', '사대부의 권력 강화 ...",2,NaN
3,augmented-3,"최근 한국의 농산물 수출이 증가하는 가운데, 특히 고품질 쌀과 김치의 수출이 두드러...",최근 한국의 농산물 수출이 증가한 주된 이유는 무엇인가?,"['고품질 쌀의 수출 증가', '유럽연합과의 무역협정 체결', '김치 수출의 감소'...",1,NaN
4,augmented-4,"20세기 초, 한국은 일본의 식민지 지배를 받으며 수많은 저항 운동이 일어났다. 특...",3.1 운동이 일어난 해는 언제인가?,"['1918년', '1919년', '1920년', '1930년', '1945년']",2,NaN
...,...,...,...,...,...,...
2229,augmented-1957,최근에는 젊은 세대가 다양한 경제적 문제를 겪고 있습니다. 특히 주택 가격의 급등과...,최근 젊은 세대가 겪고 있는 문제 중에서 주택 문제와 관련하여 정부가 추진하는 정책...,"['청년 실업과 채용 확대 문제', '소득 불균형 해소에 대한 정책', '세계 경제...",1,NaN
2230,augmented-1958,최근 한 온라인 플랫폼에서 청소년들이 '편지'라는 형식을 모른다는 이야기가 화제가 ...,청소년들이 감정을 담아 편지를 쓰지 않게 된 주된 원인은 무엇인가?,"['소통 방식의 변화와 기술 발전', '편지 쓰기의 전통적인 매력', '감사 표현의...",1,NaN
2231,augmented-1959,최근 동화 출판 분야에 신선한 바람을 일으킬 작품들이 속속 등장하고 있다. ‘202...,'2023 동화 창작 공모전'의 대표 수상작 제목은 무엇인가?,"['구름 위의 여행', '판타지의 모험과 심화', '어린이의 상상력과 꿈', '어린...",1,NaN
2232,augmented-1960,최근 들어 다양한 사회적 이슈를 담은 다큐멘터리 제작이 증가하고 있다. 다큐멘터리는...,다큐멘터리 제작에 있어 중요한 요소는 무엇인가?,"['사건의 단순 나열이 필요하다', '규칙적인 방송 시간이 중요하다', '사람의 감...",3,NaN


In [54]:
print(df1.dtypes)

id                object
paragraph         object
question          object
choices           object
answer             int64
question_plus    float64
dtype: object


In [55]:
import ast

def is_valid_choices(value):
    try:
        # 문자열을 리스트로 변환 (만약 데이터가 문자열 형태로 저장된 경우)
        if isinstance(value, str):
            value = ast.literal_eval(value)  # 안전하게 문자열을 리스트로 변환

        # 리스트 여부와 길이 조건 검증
        return isinstance(value, list) and len(value) in [4, 5]
    except (ValueError, SyntaxError):
        # 변환 오류 발생 시 False 반환
        return False

# 유효한 choices와 유효하지 않은 choices 분류
valid_df = df1[df1['choices'].apply(is_valid_choices)]
invalid_df = df1[~df1['choices'].apply(is_valid_choices)]

print("유효한 choices의 수:", len(valid_df))
print("유효하지 않은 choices의 수:", len(invalid_df))

# 유효하지 않은 choices의 예시 출력
print("\n유효하지 않은 choices의 예시:")
print(invalid_df['choices'].head())

유효한 choices의 수: 2234
유효하지 않은 choices의 수: 0

유효하지 않은 choices의 예시:
Series([], Name: choices, dtype: object)


In [44]:
# import ast

# def is_valid_choices(choices_str):
#     try:
#         choices = ast.literal_eval(choices_str)
#         return isinstance(choices, list) and 4 <= len(choices) <= 5
#     except:
#         return False

# # 유효한 choices와 유효하지 않은 choices 분류
# valid_choices = df1[df1['choices'].apply(is_valid_choices)]
# invalid_choices = df1[~df1['choices'].apply(is_valid_choices)]

# print("유효한 choices의 수:", len(valid_choices))
# print("유효하지 않은 choices의 수:", len(invalid_choices))

# # 유효하지 않은 choices의 예시 출력
# print("\n유효하지 않은 choices의 예시:")
# print(invalid_choices['choices'].head())

# # 유효한 데이터와 유효하지 않은 데이터를 별도의 데이터프레임으로 저장
# valid_df = df1[df1['choices'].apply(is_valid_choices)]
# invalid_df = df1[~df1['choices'].apply(is_valid_choices)]

In [45]:
valid_df

,id,paragraph,question,choices,answer,question_plus
0,augmented-0,“19세기말 일본은 폐쇄적인 에도 시대를 지나 메이지 유신을 통해 급속한 근대화를 ...,위 글에서 설명된 일본의 근대화 과정에서 나타난 주요 사회적 문제는 무엇으로 분석될...,"['민주주의 제도의 확립', '농민과 노동자의 경제적 불만', '전통 문화의 부흥'...",2,NaN
1,augmented-1,교육부는 최근 한국의 고등학교 교육과정 개편을 통해 '진로 탐색' 과목을 신설하겠다...,교육부가 신설 예정인 '진로 탐색' 과목의 목적은 무엇인가?,"['학생들이 다양한 직업 세계를 이해하기 위해서', '취업률을 높이기 위해서', '...",1,NaN
2,augmented-2,"“[한국사] 조선시대 중기에는 사대부와 농민 사이의 갈등이 심화되었고, 이는 다양한...",위 글에서 언급된 '대동법'의 시행 배경은 무엇과 관련이 있습니까?,"['양반의 세금 부담 증가', '농민의 경제적 안정 추구', '사대부의 권력 강화 ...",2,NaN
3,augmented-3,"최근 한국의 농산물 수출이 증가하는 가운데, 특히 고품질 쌀과 김치의 수출이 두드러...",최근 한국의 농산물 수출이 증가한 주된 이유는 무엇인가?,"['고품질 쌀의 수출 증가', '유럽연합과의 무역협정 체결', '김치 수출의 감소'...",1,NaN
4,augmented-4,"20세기 초, 한국은 일본의 식민지 지배를 받으며 수많은 저항 운동이 일어났다. 특...",3.1 운동이 일어난 해는 언제인가?,"['1918년', '1919년', '1920년', '1930년', '1945년']",2,NaN
...,...,...,...,...,...,...
2229,augmented-1957,최근에는 젊은 세대가 다양한 경제적 문제를 겪고 있습니다. 특히 주택 가격의 급등과...,최근 젊은 세대가 겪고 있는 문제 중에서 주택 문제와 관련하여 정부가 추진하는 정책...,"['청년 실업과 채용 확대 문제', '소득 불균형 해소에 대한 정책', '세계 경제...",1,NaN
2230,augmented-1958,최근 한 온라인 플랫폼에서 청소년들이 '편지'라는 형식을 모른다는 이야기가 화제가 ...,청소년들이 감정을 담아 편지를 쓰지 않게 된 주된 원인은 무엇인가?,"['소통 방식의 변화와 기술 발전', '편지 쓰기의 전통적인 매력', '감사 표현의...",1,NaN
2231,augmented-1959,최근 동화 출판 분야에 신선한 바람을 일으킬 작품들이 속속 등장하고 있다. ‘202...,'2023 동화 창작 공모전'의 대표 수상작 제목은 무엇인가?,"['구름 위의 여행', '판타지의 모험과 심화', '어린이의 상상력과 꿈', '어린...",1,NaN
2232,augmented-1960,최근 들어 다양한 사회적 이슈를 담은 다큐멘터리 제작이 증가하고 있다. 다큐멘터리는...,다큐멘터리 제작에 있어 중요한 요소는 무엇인가?,"['사건의 단순 나열이 필요하다', '규칙적인 방송 시간이 중요하다', '사람의 감...",3,NaN


In [47]:
valid_df.to_csv("normal_data2.csv", index=False)